In [1]:
!pip install sklearn_pandas
!pip install git+https://github.com/jpmml/sklearn2pmml.git

  Cloning https://github.com/jpmml/sklearn2pmml.git to /tmp/pip-q27hb18i-build


In [2]:
from sklearn2pmml import PMMLPipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn2pmml import sklearn2pmml

import pickle
import pandas as pd
# Scikit-learn method to split the dataset into train and test dataset
from sklearn.cross_validation import train_test_split
# Scikit-learn method to implement the decision tree classifier
from sklearn.tree import DecisionTreeClassifier

# Load the dataset
balance_scale_data = pd.read_csv('balancescale.data', sep=',', header=None)
print("Dataset Length:: ", len(balance_scale_data))
print("Dataset Shape:: ", balance_scale_data.shape)

# Split the dataset into train and test dataset
X = balance_scale_data.values[:, 1:5]
Y = balance_scale_data.values[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

# Decision model with Gini index critiria
decision_tree_model = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=3, min_samples_leaf=5)
pipeline = PMMLPipeline([
    ("classifier", DecisionTreeClassifier())
])
pipeline.fit(X_train, y_train)
print("Decision Tree classifier :: ", decision_tree_model)
print("Decision Tree classifier :: ", pipeline)

#print("prediction: ", decision_tree_model.predict([1,1,3,4]))
print("prediction: ", pipeline.predict([1,1,3,4]))

# Dump the trained decision tree classifier with Pickle
pipeline_pkl_filename = 'python_balancescale.pkl'

# Open the file to save as pkl file
pipeline_pkl = open(pipeline_pkl_filename, 'wb')
pickle.dump(pipeline, pipeline_pkl)
# Close the pickle instances
pipeline_pkl.close()

# Loading the saved decision tree model pickle
pipeline_pkl = open(pipeline_pkl_filename, 'rb')
pipeline = pickle.load(pipeline_pkl)
print("Loaded Decision tree model :: ", pipeline)
print("prediction: ", pipeline.predict([[1,1,3,4]]))
pipeline_pkl.close()

Dataset Length::  373
Dataset Shape::  (373, 5)
Decision Tree classifier ::  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')
Decision Tree classifier ::  PMMLPipeline(steps=[('classifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])
prediction:  ['R']
Loaded Decision tree model ::  PMMLPipeline(steps=[('classifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurit

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [6]:
sklearn2pmml(pipeline, "scikit_balancescale.pmml", with_repr = True)
pmmlBytes = bytearray(open('scikit_balancescale.pmml', 'rb').read())
print(pmmlBytes)

bytearray(b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">\n\t<Header>\n\t\t<Application name="JPMML-SkLearn" version="1.2.12"/>\n\t\t<Timestamp>2017-04-15T16:41:07Z</Timestamp>\n\t</Header>\n\t<MiningBuildTask>\n\t\t<Extension>PMMLPipeline(steps=[(\'classifier\', DecisionTreeClassifier(class_weight=None, criterion=\'gini\', max_depth=None,\n            max_features=None, max_leaf_nodes=None,\n            min_impurity_split=1e-07, min_samples_leaf=1,\n            min_samples_split=2, min_weight_fraction_leaf=0.0,\n            presort=False, random_state=None, splitter=\'best\'))])</Extension>\n\t</MiningBuildTask>\n\t<DataDictionary>\n\t\t<DataField name="y" optype="categorical" dataType="string">\n\t\t\t<Value value="B"/>\n\t\t\t<Value value="L"/>\n\t\t\t<Value value="R"/>\n\t\t</DataField>\n\t\t<DataField name="x1" optype="continuous" dataType="float"/>\n\t\t<DataField name="x2" optype="continuous" dataType="float"/>\

In [4]:
import urllib.request
import urllib.parse

update_url = "http://prediction-pmml.demo.pipeline.io/update-pmml/default/scikit_balancescale/v0"
update_headers = {}
update_headers["Content-type"] = "application/xml"
req = urllib.request.Request(update_url,headers=update_headers,data=pmmlBytes)
resp = urllib.request.urlopen(req)
print(resp.status)

200


In [5]:
import json
evaluate_url = 'http://prediction-pmml.demo.pipeline.io/evaluate-pmml/default/scikit_balancescale/v0'
evaluate_headers = {}
#evaluate_headers['Content-type'] = 'application/json'

input_params = b'{"x1":0, "x2":1, "x3":2, "x4":3}'

#encoded_input_params = input_params.encode('utf-8')
req = urllib.request.Request(evaluate_url, 
                             headers=evaluate_headers,
                             data=input_params)
resp = urllib.request.urlopen(req)
print(resp.read())

b'{"results":[{"result": "fallback"}]}'
